# Resultados Esperados
[![Resutlado Esperado](https://www.researchgate.net/profile/Albert_Serra_Pages/publication/235692900/figure/fig3/AS:299891772411907@1448511178167/LTE-turbo-code-BLER-AWGN-channel-vs-code-block-size.png)](https://www.researchgate.net/figure/BLER-curves-in-AWGN-channel-for-all-I-MCS-25-RBs_fig4_235692900)

In [1]:
import sqlite3
import inspect, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab


import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

import cufflinks as cf

from collections import OrderedDict 

init_notebook_mode(connected=True)

In [2]:
def imcs2itbs(imcs):
    return [0,2,1,3,4,5,6,7,8,9,10][imcs]

In [3]:
conn=sqlite3.connect("mono_01.db",isolation_level=None)

In [4]:
sqlquery = """
SELECT A.*
FROM   blersimu AS A /* Minimum SNR that produces the maximum estimated BLER*/
       INNER JOIN (SELECT B.imcs, B.iru, B.irep, Max(isnr) AS minisnr
                   FROM   blersimu AS B
                          INNER JOIN (SELECT imcs, iru, irep, Max(bler) AS maxbler
                                      FROM   blersimu
                                      GROUP  BY imcs,iru,irep) AS C
                                  ON B.imcs = C.imcs AND B.iru = C.iru AND B.irep = C.irep
                   WHERE  bler = maxbler
                   GROUP  BY B.imcs, B.iru, B.irep) AS D
               ON A.imcs = D.imcs AND A.iru = D.iru AND A.irep = D.irep
WHERE  bler > 0      /* BLER 0 can't be plotted on logy */
AND isnr >= minisnr  /* values above isnr are redundant */
-- AND tbs = 256
-- AND A.irep = 0
ORDER  BY A.irep, A.iru, A.imcs, A.snr DESC
""" 

In [5]:
df = pd.read_sql_query(sqlquery, conn)
markers=['.','v','^','s','p','P','X','D']

In [6]:
# for totru, totru_df in df.groupby("iru"):    
#     fig, ax = plt.subplots()
#     multi_df = totru_df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2))
#     for label, gb_df in multi_df:
#         #rint gb_df.to_string()
#         gb_df.plot(x='snr', y='bler', marker=markers[label[0]], logy=True, grid=True , ax=ax, label= ( gb_df.tbs[1] , ) + label )      
#     plt.ylim([1E-2,ax.get_ylim()[1]])
#     plt.xlim(right = totru_df.loc[ totru_df['bler']>1E-2 ]['snr'].max() + 1)
#     ax.margins(x=0)    
#     ncol= int(np.ceil(len(multi_df) / 12.0))
#     #plt.legend(title=('irep','iru','imcs'),loc="best", frameon=False , ncol=ncol)
#     plt.legend(title=('tbs','irep','iru','imcs'),loc="upper left", bbox_to_anchor=(1,1), frameon=False , ncol=ncol)
#     plt.ylabel("bler")
#     plt.title("Total Number Of Resource Units = %u" % totru)
#     plt.grid(b=True, which='minor', linestyle='--')

for totru, totru_df in df.groupby("imcs"):    
    data=[]
    for label, gb_df in multi_df:
        #rint gb_df.to_string()
        trace = go.Scatter(x=[1,2,3], y=[4,5,6], mode="markers+lines",  text=["one","two","three"], name='1st Trace')
        data.append()
        gb_df.plot(x='snr', y='bler', marker=markers[label[0]], logy=True, grid=True , ax=ax, label= ( gb_df.tbs[1] , ) + label )      

    
    fig, ax = plt.subplots()
    multi_df = totru_df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2))
    
    plt.ylim([1E-2,ax.get_ylim()[1]])
    plt.xlim(right = totru_df.loc[ totru_df['bler']>1E-2 ]['snr'].max() + 1)
    ax.margins(x=0)    
    ncol= int(np.ceil(len(multi_df) / 12.0))
    #plt.legend(title=('irep','iru','imcs'),loc="best", frameon=False , ncol=ncol)
    plt.legend(title=('tbs','irep','iru','imcs'),loc="upper left", bbox_to_anchor=(1,1), frameon=False , ncol=ncol)
    plt.ylabel("bler")
    plt.title("Total Number Of Resource Units = %u" % totru)
    plt.grid(b=True, which='minor', linestyle='--')


                                               

layout=go.Layout(title="First Plot", xaxis={'title':'x1'}, yaxis={'title':'x2'})
figure=go.Figure(data=data,layout=layout)
iplot(figure, filename='pyguide_1')

# Data grouped by IRU

In [7]:
for totru, totru_df in df.groupby("iru"):
    data=[]
    multi_df = totru_df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2))
    for label, gb_df in multi_df:
        trace = go.Scatter(x=gb_df['snr'], y=gb_df['bler'], mode="markers+lines", name=str( ( gb_df.tbs.iloc[0] , ) + label), showlegend=True)
        data.append(trace)                
    

    annotations=[
        dict(
            x=1.12,
            y=1.05,
            align="right",
            valign="top",
            text="tbs,irep,iru,imcs",
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top"
        )]
    layout=go.Layout(title="IRU:%i" % label[1], xaxis={'title':'SNR'}, yaxis={'title':'BLER','type':'log','range':[-2,0.301],'gridwidth':2,},annotations=annotations)


    figure=go.Figure(data=data,layout=layout)
    iplot(figure, filename="IRU_%i" % label[1])    

# Data grouped by IMCS

In [8]:
for totru, totru_df in df.groupby("imcs"):
    data=[]
    multi_df = totru_df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2))
    for label, gb_df in multi_df:
        trace = go.Scatter(x=gb_df['snr'], y=gb_df['bler'], mode="markers+lines", name=str( ( gb_df.tbs.iloc[0] , ) + label), showlegend=True)
        data.append(trace)                
    

    annotations=[
        dict(
            x=1.12,
            y=1.05,
            align="right",
            valign="top",
            text="tbs,irep,iru,imcs",
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top"
        )]
    layout=go.Layout(title="IMCS:%i" % label[1], xaxis={'title':'SNR'}, yaxis={'title':'BLER','type':'log','range':[-2,0.301],'gridwidth':2,},annotations=annotations)


    figure=go.Figure(data=data,layout=layout)
    iplot(figure, filename="IMCS_%i" % label[1])    

## All curves

In [9]:
markers=['.','v','^','s','p','P','X','D']
data=[]
for totru, totru_df in df.groupby("imcs"):    
    multi_df = totru_df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2))
    for label, gb_df in multi_df:
        #trace = go.Scatter(x=gb_df['snr'], y=gb_df['bler'], mode="markers+lines", name=str( ( gb_df.tbs.iloc[0] , ) + label), showlegend=True)
        trace = go.Scatter(x=gb_df['snr'], y=gb_df['bler'], mode="markers+lines", name=str(label), showlegend=True)
        data.append(trace)                
    

    annotations=[
        dict(
            x=1.12,
            y=1.05,
            align="right",
            valign="top",
            text="irep,iru,imcs",
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top"
        )]

layout=go.Layout(title="IMCS:%i" % label[1],
                 autosize=False,
                 height=413,
                 width=1000,
                 xaxis={'title':'SNR'}, 
                 yaxis={'title':'BLER','type':'log','range':[-2,0.301],'gridwidth':2,},
                 annotations=annotations,
                 
                )
figure=go.Figure(data=data,layout=layout)
iplot(figure, filename="IMCS_%i" % label[1])   

In [10]:
#Codigo para plotear markers
# data=[]

# for x in range(33):
#     trace = go.Scatter( x=[0,1], y=[x,x], marker={'symbol' : x}, mode="markers+lines", showlegend=True )
#     data.append(trace)
    
# figure=go.Figure(data=data)
# iplot(figure, filename="IMCS_%i" % label[1])

In [73]:
markers=[0,1,2,4,13,17,18,21] #Ver https://www.r-graph-gallery.com/125-the-plotlys-symbols/

# cmap={1:'rgb(240,249,232)',
#       2:'rgb(204,235,197)',
#       3:'rgb(168,221,181)',
#       4:'rgb(123,204,196)',
#       5:'rgb(78,179,211)',
#       6:'rgb(43,140,190)',
#       7:'rgb(8,88,158)'}

# cmap={1:'rgb(255,255,178)',
#       2:'rgb(254,217,118)',
#       3:'rgb(254,178,76)',
#       4:'rgb(253,141,60)',
#       5:'rgb(252,78,42)',
#       6:'rgb(227,26,28)',
#       7:'rgb(177,0,38)'}

cmap={1:'rgb(148,0,211)',
      2:'rgb(75,0,130)',
      3:'rgb(0,0,255)',
      4:'rgb(0,255,0)',
      5:'rgb(255,255,0)',
      6:'rgb(255,127,0)',
      7:'rgb(255,0,0)'}

data=[]

#for totru, totru_df in df.groupby("irep"):        
indexed=df.set_index(['irep','iru','imcs','isnr'])
for label, gb_df in indexed.groupby(level=(0,1,2)):    
    trace = go.Scatter(x=gb_df['snr'], y=gb_df['bler'], marker={'symbol' : markers[label[0]], 'color':cmap[label[1]]}, mode="markers+lines", name=str(label), showlegend=False)
    data.append(trace)                
    

#     annotations=[
#         dict(
#             x=1.12,
#             y=1.05,
#             align="right",
#             valign="top",
#             #text="irep,iru,imcs",
#             showarrow=False,
#             xref="paper",
#             yref="paper",
#             xanchor="center",
#             yanchor="top"
#         )]


for label, gb_df in  indexed.groupby(level=(0)):    
    trace = go.Scatter(x=[ gb_df['snr'].iloc[0], ], y=[ gb_df['bler'].iloc[0] ,], marker={'symbol' : markers[label] ,'color':'black'}, mode="markers", name=str(label), visible='legendonly',showlegend=True, legendgroup='IREP')
    data.append(trace)                

for label, gb_df in  indexed.groupby(level=(1,2)):    
    trace = go.Scatter(x=[ gb_df['snr'].iloc[0], ], y=[ gb_df['bler'].iloc[0] ,], marker={'color':cmap[label[0]]}, mode="lines", name=str(label), visible='legendonly',showlegend=True, legendgroup='(IMCS,IRU)')
    data.append(trace)    
#print [ x[0] for x in indexed.groupby(level=(0)) ]
#print [ x[0] for x in indexed.groupby(level=(1,2)) ]

layout=go.Layout(autosize=False,
                 height=313,
                 width=1000,
                 margin={'l':80, 'r':200, 't':20, 'b':100},
                 xaxis={'title':'SNR','range':[-30,6]}, 
                 yaxis={'title':'BLER','type':'log','range':[-2,0.01],'gridwidth':2,},
                 legend={'orientation':'h', 'xanchor' : 'left', 'x':1, 'y':1 },
#                 legend={'orientation' : 'h', },
#                  annotations=annotations,                 
                )
figure=go.Figure(data=data,layout=layout)
iplot(figure, filename="alugn titulo")   

In [12]:
pio.write_image(figure, '/tmp/todas.pdf')

## Clusters

In [13]:
iru2nru=[1,2,3,4,5,6,8,10]
blerth=0.1
data=[]
#multi_df = df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2))
#for label, gb_df in multi_df:    
#    lim_snr=np.interp(blerth, gb_df['bler'].values, gb_df['snr'].values)
#    data.append((label[0],label[1],label[2],iru2nru[label[1]] * (2** label[0]), lim_snr) ) 

#chose_df = pd.DataFrame(data, columns=['irep','iru','imcs','totru','snrth']).sort_values('snrth',ascending=False)


In [14]:

def _groupProcess(x,blerth):
    result=OrderedDict()
    result['totru']=x['totru'].values[0]
    result['snrth']=np.interp(blerth, x['bler'].values, x['snr'].values) 
    
    #Find bounds
    
    ub=x.loc[x.loc[x['bler']>=blerth,'bler'].idxmin()]    
    result['ubsnr']=ub['snr']
    result['ubbler']=ub['bler']
    
    lb=x.loc[x.loc[x['bler']<=blerth,'bler'].idxmax()]      
    result['lbsnr']=lb['snr']
    result['lbbler']=lb['bler']
    
    return pd.Series(result)

thresholds=df.set_index(['irep','iru','imcs','isnr']).groupby(level=(0,1,2)).apply(lambda x: _groupProcess(x,0.1))
#Filter Duplicated and add sort index
thresholds=thresholds.loc[thresholds.groupby('totru' ,as_index=False)['snrth'].idxmax()]

#Add delta
thresholds.insert(thresholds.columns.get_loc('snrth')+1,
                  'delta', thresholds.shift(1)['snrth']-thresholds['snrth'])
#Remove NaN
thresholds['delta'].fillna(10, inplace=True)
thresholds

totru      snrth      delta  ubsnr    ubbler  lbsnr    lbbler
irep iru imcs                                                                
0    1   8        2.0   4.465357  10.000000    4.4  0.121521    4.5  0.088593
     2   6        3.0   1.407053   3.058304    1.4  0.102417    1.5  0.068146
     3   4        4.0  -0.288218   1.695271   -0.3  0.104156   -0.2  0.068878
     4   3        5.0  -1.382066   1.093848   -1.4  0.106781   -1.3  0.068970
     5   1        6.0  -1.602262   0.220196   -1.7  0.144562   -1.6  0.098969
2    1   8        8.0  -3.313611   1.711349   -3.4  0.140442   -3.3  0.093628
0    7   0       10.0  -4.398701   1.085089   -4.4  0.100433   -4.3  0.067078
2    2   6       12.0  -5.119158   0.720457   -5.2  0.136932   -5.1  0.091248
3    1   8       16.0  -6.325123   1.205965   -6.4  0.133362   -6.3  0.088806
2    4   3       20.0  -7.420473   1.095349   -7.5  0.136963   -7.4  0.090485
     5   1       24.0  -8.129170   0.708698   -8.2  0.129700   -8.1  0.087769
4    1   8       32.0  -9.326537   1.197367   -9.4  0.128876   -9.3  0.089569
3    4   3       40.0 -10.429082   1.102545  -10.5  0.131836  -10.4  0.086945
4    2   6       48.0 -11.136575   0.707492  -11.2  0.127350  -11.1  0.084229
5    1   8       64.0 -12.340274   1.203700  -12.4  0.125244  -12.3  0.082977
4    4   3       80.0 -13.439082   1.098808  -13.5  0.122681  -13.4  0.085449
5    2   6       96.0 -14.150926   0.711844  -14.2  0.119080  -14.1  0.080200
6    1   8      128.0 -15.351016   1.200090  -15.4  0.120300  -15.3  0.078857
4    7   0      160.0 -16.447714   1.096698  -16.5  0.121429  -16.4  0.080444
5    5   1      192.0 -17.161641   0.713927  -17.2  0.114410  -17.1  0.076843
7    1   8      256.0 -18.360657   1.199016  -18.4  0.115356  -18.3  0.076324
5    7   0      320.0 -19.460988   1.100331  -19.5  0.113739  -19.4  0.078522
7    2   6      384.0 -20.176348   0.715360  -20.2  0.109131  -20.1  0.070526
     3   4      512.0 -21.487527   1.311179  -21.5  0.104187  -21.4  0.070618
     4   3      640.0 -22.469946   0.982419  -22.5  0.110199  -22.4  0.076263
     5   1      768.0 -23.183011   0.713065  -23.2  0.105957  -23.1  0.070892
     6   2     1024.0 -24.535478   1.352467  -24.6  0.125519  -24.5  0.085968
     7   0     1280.0 -25.483137   0.947659  -25.5  0.106201  -25.4  0.069427

In [15]:
with open('/tmp/test2.html','w') as f:
    f.write(thresholds.to_html())

In [16]:
bounded=thresholds.copy()
bounded.insert(thresholds.columns.get_loc('snrth')+1,
                  'rbound', thresholds.shift(1)['snrth'])
bounded.insert(thresholds.columns.get_loc('snrth')+1,
                  'abound', thresholds.shift(1)['snrth'])
bounded['rbound'].fillna(1000, inplace=True)
bounded['abound'].fillna(10, inplace=True)

In [17]:
data=[]

for index,row in bounded.iterrows():
    trace = go.Scatter(
    x=[row['snrth'],row['rbound']],
    y=[row['totru'],row['totru']],
    fill='tozeroy',
    name=str(index),
    showlegend=False,
    hoveron = 'points',
    hoverinfo= 'text',
    text=str(index)
    )
    data.append(trace)

layout=go.Layout(title="boundaries", xaxis={'title':'SNR','range':[-26,10],}, yaxis={'title':'totru','type':'log','gridwidth':2,})
figure=go.Figure(data=data,layout=layout)

    
iplot(figure, filename="boundaries", colorscale='Reds')   

TypeError: iplot() got an unexpected keyword argument 'colorscale'

In [ ]:
data=[]
annotations = []

for index,row in bounded.iterrows():
    trace = go.Scatter(
    x=[row['snrth'],row['rbound']],
    y=[row['totru'],row['totru']],
    fill='tozeroy',
    name=str(index),
    showlegend=False,    
    mode='lines',
    )
    data.append(trace)
    
    
    annotation=dict(x=(row['snrth']+row['abound'])/2,
                    y=np.log10(row['totru']),
                    text='<b>%s</b>' % str(index),
                    font=dict(family='Arial', size=10,),
                    showarrow=False,
                    textangle = -45,
                    yanchor = "bottom",
                    xanchor = "left"
                   
                   )

    annotations.append(annotation)

    
annotation=dict(    x=0.75,y=0.75,
                    text='<b>(IREP,IMCS,IRU)</b>',
                    font=dict(family='Arial', size=15,),
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    yanchor = "bottom",
                    xanchor = "left",
                   )
    
annotations.append(annotation)
    
layout=go.Layout(title="boundaries", xaxis={'title':'SNR','range':[-26,10],'zeroline': False,}, yaxis={'title':'RUs','type':'log','gridwidth':2, },annotations=annotations)
figure=go.Figure(data=data,layout=layout)

    
iplot(figure, filename="boundaries")   

In [ ]:
pio.write_image(figure, '/tmp/fig1.pdf')

In [ ]:
data=[]
annotations = []

cmap=[(0x67,0x00,0x0d),
(0xbc,0x14,0x19),
(0xf2,0x44,0x32),
#(0xfc,0x8a,0x6a),
#(0xff,0xf5,0xf0),
]

i=0
for index,row in bounded.iterrows():    
    trace = go.Scatter(
    x=[row['snrth'],row['rbound']],
    y=[row['totru'],row['totru']],
    fill='tozeroy',
    name=str((imcs2itbs(index[1]),2**index[0])),
    showlegend=False,    
    mode='lines',         
    marker=dict(color='rgb(%i,%i,%i)'% cmap[i%len(cmap)])
    )    
    data.append(trace)
    
    
    annotation=dict(x=(row['snrth']+row['abound'])/2,
                    y=np.log10(row['totru']),
                    text='<b>%s</b>' % str((imcs2itbs(index[1]),2**index[0])),
                    font=dict(family='Arial', size=10,),f
                    showarrow=False,
                    textangle = -45,
                    yanchor = "bottom",
                    xanchor = "left"
                   
                   )
    i=i+1
    annotations.append(annotation)

    
annotation=dict(    x=0.75,y=0.75,
                    text='<b>(I<sub>TBS</sub>,NR)</b>',
                    font=dict(family='Arial', size=15,),
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    yanchor = "bottom",
                    xanchor = "left",
                   )
    
annotations.append(annotation)
    
layout=go.Layout(title="boundaries", xaxis={'title':'SNR','range':[-26,10],'zeroline': False,}, yaxis={'title':'RUs','type':'log','gridwidth':2, },annotations=annotations)
figure=go.Figure(data=data,layout=layout)

    
iplot(figure, filename="boundaries")   

In [ ]:
pio.write_image(figure, '/tmp/fig1.pdf')